In [ ]:
import os
import pandas as pd
import numpy as np

# Paths of the files to be used for formatting. (Specify with extension)
# Insert the paths between the quotes
IMPORT_FILE = r'D:\IMP_2022.csv'
EXPORT_FILE = r'D:\EXP_2022.csv'

# Folder where the files will be saved.
EXPORT_DIRECTORY = r'C:\Users\Eckesaht\Desktop\as'

# Reading the data from CSV files
imp = pd.read_csv(IMPORT_FILE, sep=';')
exp = pd.read_csv(EXPORT_FILE, sep=';')
os.makedirs(EXPORT_DIRECTORY, exist_ok=True)

# Renaming the column 'CO_NCM'
MAPPING = {'CO_NCM': 'NCM'}
imp = imp.rename(columns=MAPPING)
exp = exp.rename(columns=MAPPING)

# Dropping unwanted columns
DESIRED_COLUMNS = ['NCM', 'VL_FOB', 'CO_MES', 'SG_UF_NCM']
REMOVED_COLUMNS_imp = [column for column in imp.columns if column not in DESIRED_COLUMNS]
REMOVED_COLUMNS_exp = [column for column in exp.columns if column not in DESIRED_COLUMNS]
imp = imp.drop(columns=REMOVED_COLUMNS_imp)
exp = exp.drop(columns=REMOVED_COLUMNS_exp)

# Transforming the 'SG_UF_NCM' column into unique values, avoiding repetitions per month
ufs_imp = imp['SG_UF_NCM'].unique()
ufs_exp = exp['SG_UF_NCM'].unique()
UFS = set(ufs_imp).union(ufs_exp)

# Transforming the 'CO_MES' column into columns representing the months
imp['CO_MES'] = imp['CO_MES'].replace({
    1: 'Imp_jan',
    2: 'Imp_fev',
    3: 'Imp_mar',
    4: 'Imp_abr',
    5: 'Imp_mai',
    6: 'Imp_jun',
    7: 'Imp_jul',
    8: 'Imp_ago',
    9: 'Imp_set',
    10: 'Imp_out',
    11: 'Imp_nov',
    12: 'Imp_dez'
})

exp['CO_MES'] = exp['CO_MES'].replace({
    1: 'Exp_jan',
    2: 'Exp_fev',
    3: 'Exp_mar',
    4: 'Exp_abr',
    5: 'Exp_mai',
    6: 'Exp_jun',
    7: 'Exp_jul',
    8: 'Exp_ago',
    9: 'Exp_set',
    10: 'Exp_out',
    11: 'Exp_nov',
    12: 'Exp_dez'
})

# Transforming months into columns
for uf in UFS:
    filtered_uf_imp = imp[imp['SG_UF_NCM'] == uf]
    filtered_uf_exp = exp[exp['SG_UF_NCM'] == uf]

    filtered_uf_imp = filtered_uf_imp.pivot_table(index=['NCM', 'SG_UF_NCM'], columns='CO_MES', values='VL_FOB', aggfunc='sum').reset_index()
    filtered_uf_exp = filtered_uf_exp.pivot_table(index=['NCM', 'SG_UF_NCM'], columns='CO_MES', values='VL_FOB', aggfunc='sum').reset_index()

    specified_order = ['NCM', 'Imp_jan', 'Imp_fev', 'Imp_mar', 'Imp_abr', 'Imp_mai', 'Imp_jun', 'Imp_jul', 'Imp_ago', 'Imp_set', 'Imp_out', 'Imp_nov', 'Imp_dez']
    filtered_uf_imp = filtered_uf_imp.reindex(columns=specified_order)

    specified_order = ['NCM', 'Exp_jan', 'Exp_fev', 'Exp_mar', 'Exp_abr', 'Exp_mai', 'Exp_jun', 'Exp_jul', 'Exp_ago', 'Exp_set', 'Exp_out', 'Exp_nov', 'Exp_dez']
    filtered_uf_exp = filtered_uf_exp.reindex(columns=specified_order)

    # Merging the two files into a single file
    merged_sheet = pd.merge(filtered_uf_imp, filtered_uf_exp, on=['NCM'], how='outer')

    # Organizing columns in the right order
    correct_order = ['NCM', 'Exp_jan', 'Imp_jan', 'Net_jan', 'Exp_fev', 'Imp_fev', 'Net_fev', 'Exp_mar', 'Imp_mar', 'Net_mar', 'Exp_abr', 'Imp_abr', 'Net_abr', 'Exp_mai', 'Imp_mai', 'Net_mai', 'Exp_jun', 'Imp_jun', 'Net_jun', 'Exp_jul', 'Imp_jul', 'Net_jul', 'Exp_ago', 'Imp_ago', 'Net_ago', 'Exp_set', 'Imp_set', 'Net_set', 'Exp_out', 'Imp_out', 'Net_out', 'Exp_nov', 'Imp_nov', 'Net_nov', 'Exp_dez', 'Imp_dez', 'Net_dez', 'Exp_2022', 'Imp_2022', 'Net_2022']
    merged_sheet = merged_sheet.reindex(columns=correct_order)

    # Assigning column names to be summed to three constants
    months = ['jan', 'fev', 'mar', 'abr', 'mai', 'jun', 'jul', 'ago', 'set', 'out', 'nov', 'dez']
    for month in months:
        NET_COLUMN = 'Net_' + month
        EXP_COLUMN = 'Exp_' + month
        IMP_COLUMN = 'Imp_' + month

        # Checking for empty cells
        EXP_NULL = merged_sheet[EXP_COLUMN].isnull()
        IMP_NULL = merged_sheet[IMP_COLUMN].isnull()
        EXP_OR_IMP_NULL = np.logical_or(EXP_NULL, IMP_NULL)

        # Calculating EXP - IMP and assigning the result to the NET cell
        merged_sheet[NET_COLUMN] = np.where(EXP_NULL, -merged_sheet[IMP_COLUMN], np.where(IMP_NULL, merged_sheet[EXP_COLUMN], merged_sheet[EXP_COLUMN] - merged_sheet[IMP_COLUMN]))

    # Summing all the numbers of the cells and assigning the results to their respective last lines
    merged_sheet['Exp_2022'] = merged_sheet.filter(like='Exp_').sum(axis=1)
    merged_sheet['Imp_2022'] = merged_sheet.filter(like='Imp_').sum(axis=1)
    merged_sheet['Net_2022'] = merged_sheet.filter(like='Net_').sum(axis=1)

    # Exporting the file to CSV format
    PATH = os.path.join(EXPORT_DIRECTORY, f'{uf}.csv')
    merged_sheet.to_csv(PATH, index=False, sep=',')
